In [1]:
from utils import DateSet, numpy2gpu,load_split_data
from utils import gen_tests_of_deblur,x_post_sample_modify,prior_mean_cov
from utils import sample_cumulated_sum
from function import get_x_ml
from visualize import printdict
%matplotlib inline

In [2]:
%run problem_setting.py

sigma_range: [0.001, 0.1]
noise_num: 5000
M_samples_per_para: 40
x_dim: 50
data_dim: 50
sigma_prior: 0.01
    H: (50, 50)
data_file_prefix: data\signal_denoise_mu5000_0.001_0.1_M40
---------------------------------------------------------------------------
number of samples: 5000*  40  (noise_num*M_samples_per_para)
---------------------------------------------------------------------------


# load model and simulate test data 

In [3]:
model_file_name = 'type1_12_14_ls5_lr4e-06' #??

args = np.load(os.path.join('saved_model',model_file_name)+"_args.npy",allow_pickle=True).item()
hypers = np.load(os.path.join('saved_model',model_file_name)+"_hypers.npy",allow_pickle=True).item() ##############??

print(model_file_name)
printdict(args),printdict(hypers)

cvae = torch.load(os.path.join('saved_model',model_file_name)+".pth") #??

type1_12_14_ls5_lr4e-06
epochs: 10
batch_size: 64
encoder_layer_sizes: [101, 60, 100, 300, 700, 1400, 4000]
decoder_layer_sizes: [4000, 1400, 700, 300, 100, 60, 10, 1]
latent_size: 5
print_every: 5000
fig_root: figs
conditional: True
sigma_range: [0.001, 0.1]
noise_num: 5000
M_samples_per_para: 40
x_dim: 50
data_dim: 50
sigma_prior: 0.01
    H: (50, 50)
data_file_prefix: data/signal_denoise_mu5000_0.001_0.1_M40


# 函数定义

In [4]:
def get_x_ml(y,hypers):
    """
    max p(y|x)
    """
    # y is  a ndarray
    n,m = hypers['data_dim'],hypers['x_dim']
    data = y.flatten()
    # Construct the problem.

    x = cp.Variable([m,1])
    f = cp.sum_squares(data-x.flatten())
    objective = cp.Minimize(f)
    p = cp.Problem(objective)

    # Assign a value to gamma and find the optimal x.
    result = p.solve(solver=cp.SCS,verbose=False)
    return (x.value).reshape(-1,1)

In [5]:
def x_post_sample_modify(m,sigma_prior,sigma_noise,y):
    I=np.identity(m)
    T_pr_inv = sigma_prior**(-2) * I
    T_obs_inv = sigma_noise**(-2) * I
    post_cov = np.linalg.inv(T_pr_inv+T_obs_inv)
    post_mean = post_cov@(T_obs_inv@y)
    
    post_cov_L = np.linalg.cholesky(post_cov)
    x_post_sample = post_mean+post_cov_L@np.random.randn(m,1)
    return x_post_sample,post_mean,post_cov

# 产生真实数据

In [81]:
x_dim = hypers['x_dim']
n = 20 # 一共有10个数据
C = np.identity(x_dim) * (hypers['sigma_prior']**2)
x = np.random.multivariate_normal([0] * x_dim, C,n) # [M,x_dim]

In [85]:
x = pd.DataFrame(x)
# save = pd.DataFrame(np_data, columns = ['year', 'month', 'day']) 
x.to_csv('D://LiulanqiDownload//jupyter code//signal_denoise//x.csv',index=False,header=False)  #index=False,header=False表示不保存行索引和列标题

In [91]:
x = pd.read_csv('D://LiulanqiDownload//jupyter code//signal_denoise//x.csv',header=None)

In [92]:
x = np.array(x)

In [67]:
hypers['sigma_noise'] = 0.08
sigma_noise = hypers['sigma_noise']
np.random.seed(10)
Hx = x
b = np.random.randn(n,50) * sigma_noise
y = Hx + b

In [ ]:
y = pd.DataFrame(x)
# save = pd.DataFrame(np_data, columns = ['year', 'month', 'day']) 
y.to_csv('D://LiulanqiDownload//jupyter code//signal_denoise//y.csv',index=False,header=False) 

# 没有标准化的版本

In [84]:
for sigma_true in [0.01,0.03,0.05,0.08]:
    hypers['sigma_noise'] = sigma_true
    sigma_noise = hypers['sigma_noise']
    np.random.seed(0)
    Hx = x
    b = np.random.randn(n,50) * sigma_noise
    y = Hx + b
    
    THETA = []
    RMSE = []
    X = []
    T = []
    init=[]
    
    
    for j in tqdm(range(n)):    
        # def Gibbs_sample(N,use_sample_size,x_truth,y,hypers,show_step=200,theta_truth=None):
        N = 1000
        use_sample_size = 500
        x_truth = x[j].reshape(-1,1)
        theta_truth = np.zeros(1)#??
        m = hypers['x_dim']
        y_j = y[j]

        x_0 =  get_x_ml(y_j,hypers)
        x_init_value = x_0
        init.append(x_init_value)
        
        x_sum,x_square_sum = np.zeros_like(x_truth),np.zeros_like(x_truth)
        theta_sum,theta_square_sum = np.zeros_like(theta_truth),np.zeros_like(theta_truth)  

        z = torch.randn([1, args['latent_size']]).to(device)
        for i in range(N):
            # sample theta
            c = torch.cat((torch.tensor(x_0).float(),torch.tensor(y_j.reshape(-1,1)).float()),dim=0).T.to(device)
            theta_0_gpu = cvae.inference(z, c=c)
            theta_0 = theta_0_gpu.detach().cpu().numpy()[0]
          # 因为之前在训练网络的时候，训练数据的时候我们先标准化数据了。这个时候需要标准化回来。乘以的也是原本训练时候的均值与方差
            # sample x
            sigma = theta_0
            T.append(sigma)

            x_0,_,_ = x_post_sample_modify(m,hypers['sigma_prior'],sigma,y_j.reshape(-1,1))
            X.append(np.sqrt(((x_0 - x_truth)**2).sum()/50))

            if i>(N-use_sample_size):
                theta_sum,theta_square_sum = sample_cumulated_sum(theta_sum,theta_square_sum,theta_0)
                x_sum,x_square_sum = sample_cumulated_sum(x_sum,x_square_sum,x_0)   

        x_mean = x_sum/use_sample_size
        x_var = x_square_sum/use_sample_size-x_mean**2

        theta_mean = theta_sum/use_sample_size
        theta_var = theta_square_sum/use_sample_size-theta_mean**2
        rmse = np.sqrt(((x_mean - x_truth)**2).sum()/50)

        THETA.append(theta_mean)
        RMSE.append(rmse)
    
    noise_mean = np.array(THETA).mean()
    rmse_mean = np.array(RMSE).mean()
    
    
    print('[CVAE_within_Gibbd] noise_gt: %.2f | rmse %.4f, noise_est: %.4f' % (
            sigma_true, rmse_mean, noise_mean
        ))



100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:11<00:00,  3.59s/it]


[CVAE_within_Gibbd] noise_gt: 0.01 | rmse 0.0088, noise_est: 0.0267


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:11<00:00,  3.57s/it]


[CVAE_within_Gibbd] noise_gt: 0.03 | rmse 0.0094, noise_est: 0.0293


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:11<00:00,  3.57s/it]


[CVAE_within_Gibbd] noise_gt: 0.05 | rmse 0.0102, noise_est: 0.0324


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:11<00:00,  3.57s/it]

[CVAE_within_Gibbd] noise_gt: 0.08 | rmse 0.0111, noise_est: 0.0369


In [83]:
THETA

[array([0.03340898]),
 array([0.02791102]),
 array([0.0433258]),
 array([0.03114116]),
 array([0.04009191]),
 array([0.02717631]),
 array([0.04395394]),
 array([0.0455601]),
 array([0.02669987]),
 array([0.02897704]),
 array([0.0399254]),
 array([0.0436222]),
 array([0.03271861]),
 array([0.03485844]),
 array([0.0344599]),
 array([0.04069585]),
 array([0.04225967]),
 array([0.0459307]),
 array([0.03835168]),
 array([0.03783121])]

In [89]:
for i in range(50):
    # sample theta
    linshi = torch.randn([50,1])
    m = torch.cat((torch.tensor(linshi).float(),torch.tensor(y[0].reshape(-1,1)).float()),dim=0).T.to(device)
    print(cvae.inference(z, c=m))

tensor([[0.0397]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0411]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0556]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0477]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0452]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0286]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0349]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0514]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0469]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0373]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0428]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0468]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0533]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0527]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0420]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0542]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0357]], devic

C:\Users\yang\AppData\Local\Temp/ipykernel_20752/39504917.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  m = torch.cat((torch.tensor(linshi).float(),torch.tensor(y[0].reshape(-1,1)).float()),dim=0).T.to(device)


In [70]:
np.array(THETA).mean() # 0.08

0.03302641041260213

# AGEM signal denoise

In [93]:
from AGEMsolver import solve_agem
import numpy as np

def run_test():
    x_dim = 50
    sigma_prior = hypers['sigma_prior'] # 对应新方法的 sigma_prior
    sigma_proposal = 0.01
    sigma_noise_hat_init = 0.01 # 初始项
    cov = np.identity(x_dim) * sigma_prior**2
    x_test = x
    x_true = x_test[:]

    noise_shape = x_true.shape[1:]
    n_dim = np.prod(noise_shape)


    for noise in [0.01,0.03,0.05,0.08]: #??   # 对应sigma_noise ，为真实的噪音取值！
        sigma_noise = [noise] * n_dim
        sigma_noise = np.array(sigma_noise[:n_dim]).reshape(noise_shape)

        rmse_mean, rmse_std, noise_mean, noise_std, variance_noise_hat_em = \
            solve_agem(x_true=x_true, sigma_noise=sigma_noise,
                       sigma_prior=sigma_prior, sigma_noise_hat_init=sigma_noise_hat_init,
                       sigma_proposal=sigma_proposal, type_proposal='mala',
                       candidate='mean', em_epochs=10, sample_epochs=1000)

        print('[AGEM] noise_gt: %.2f | rmse %.4f (%.4f), noise_est: %.4f (%.4f)' % (
            noise, rmse_mean, rmse_std, noise_mean, noise_std
        ))


if __name__ == '__main__':
    run_test()

epoch 0, time 0.2 sec | noise_gt: 0.0100, noise_est: 0.0112 (0.0007) | rmse: 0.0119 (0.0016)
epoch 1, time 0.2 sec | noise_gt: 0.0100, noise_est: 0.0118 (0.0009) | rmse: 0.0140 (0.0009)
epoch 2, time 0.2 sec | noise_gt: 0.0100, noise_est: 0.0119 (0.0011) | rmse: 0.0143 (0.0007)
epoch 3, time 0.2 sec | noise_gt: 0.0100, noise_est: 0.0119 (0.0012) | rmse: 0.0144 (0.0010)
epoch 4, time 0.2 sec | noise_gt: 0.0100, noise_est: 0.0122 (0.0014) | rmse: 0.0145 (0.0009)
epoch 5, time 0.2 sec | noise_gt: 0.0100, noise_est: 0.0123 (0.0014) | rmse: 0.0146 (0.0008)
epoch 6, time 0.1 sec | noise_gt: 0.0100, noise_est: 0.0124 (0.0015) | rmse: 0.0147 (0.0009)
epoch 7, time 0.1 sec | noise_gt: 0.0100, noise_est: 0.0124 (0.0013) | rmse: 0.0146 (0.0009)
epoch 8, time 0.2 sec | noise_gt: 0.0100, noise_est: 0.0125 (0.0012) | rmse: 0.0144 (0.0010)
epoch 9, time 0.1 sec | noise_gt: 0.0100, noise_est: 0.0124 (0.0012) | rmse: 0.0144 (0.0011)
[AGEM] noise_gt: 0.01 | rmse 0.0144 (0.0011), noise_est: 0.0124 (0.001

D:\LiulanqiDownload\jupyter code\signal_denoise\infovae\无条件、复杂\AGEMsolver.py:102: RuntimeWarning: overflow encountered in exp
  replace = np.random.uniform(0, 1, x.shape[0]) < np.exp(log_alpha)


epoch 1, time 0.1 sec | noise_gt: 0.0800, noise_est: 0.0363 (0.0037) | rmse: 0.0452 (0.0032)
epoch 2, time 0.1 sec | noise_gt: 0.0800, noise_est: 0.0363 (0.0037) | rmse: 0.0452 (0.0032)
epoch 3, time 0.2 sec | noise_gt: 0.0800, noise_est: 0.0363 (0.0037) | rmse: 0.0452 (0.0032)
epoch 4, time 0.2 sec | noise_gt: 0.0800, noise_est: 0.0363 (0.0037) | rmse: 0.0452 (0.0032)
epoch 5, time 0.1 sec | noise_gt: 0.0800, noise_est: 0.0363 (0.0037) | rmse: 0.0452 (0.0032)
epoch 6, time 0.2 sec | noise_gt: 0.0800, noise_est: 0.0363 (0.0037) | rmse: 0.0452 (0.0032)
epoch 7, time 0.1 sec | noise_gt: 0.0800, noise_est: 0.0363 (0.0037) | rmse: 0.0452 (0.0032)
epoch 8, time 0.2 sec | noise_gt: 0.0800, noise_est: 0.0363 (0.0037) | rmse: 0.0452 (0.0032)
epoch 9, time 0.2 sec | noise_gt: 0.0800, noise_est: 0.0363 (0.0037) | rmse: 0.0452 (0.0032)
[AGEM] noise_gt: 0.08 | rmse 0.0452 (0.0032), noise_est: 0.0363 (0.0037)


# 求解信号去噪问题

In [19]:
mean,std = 0.0505, 0.028 #？？

c_mean = torch.mean(torch.cat((torch.tensor(x).float(),torch.tensor(y).float()),dim=1),dim=0,keepdim=True)
c_std = torch.std(torch.cat((torch.tensor(x).float(),torch.tensor(y).float()),dim=1),dim=0,keepdim=True)

In [20]:
THETA = []
RMSE = []
for j in range(n):    
    # def Gibbs_sample(N,use_sample_size,x_truth,y,hypers,show_step=200,theta_truth=None):
    N = 1000
    use_sample_size = 500
    x_truth = x[j].reshape(-1,1)
    theta_truth = np.zeros(1)#??
    m = hypers['x_dim']
    y_j = y[j]

    x_0 =  get_x_ml(y_j,hypers)
    x_init_value = x_0
    np.random.seed(3)    
    x_sum,x_square_sum = np.zeros_like(x_truth),np.zeros_like(x_truth)
    theta_sum,theta_square_sum = np.zeros_like(theta_truth),np.zeros_like(theta_truth)  

    z = torch.randn([1, args['latent_size']]).to(device)
    for i in tqdm(range(N)):
        # sample theta
        c = torch.cat((torch.tensor(x_0).float(),torch.tensor(y_j.reshape(-1,1)).float()),dim=0).T.to(device)
        c_norm = (c-c_mean.to(device))/c_std.to(device)
        theta_0_gpu = cvae.inference(z, c=c_norm)
        theta_0 = theta_0_gpu.detach().cpu().numpy()[0]
      # 因为之前在训练网络的时候，训练数据的时候我们先标准化数据了。这个时候需要标准化回来。乘以的也是原本训练时候的均值与方差
        theta_0 = theta_0*std + mean
        # sample x
        sigma = theta_0
        
        x_0,_,_ = x_post_sample_modify(m,hypers['sigma_prior'],sigma,y_j.reshape(-1,1))

        if i>(N-use_sample_size):
            theta_sum,theta_square_sum = sample_cumulated_sum(theta_sum,theta_square_sum,theta_0)
            x_sum,x_square_sum = sample_cumulated_sum(x_sum,x_square_sum,x_0)   

    x_mean = x_sum/use_sample_size
    x_var = x_square_sum/use_sample_size-x_mean**2

    theta_mean = theta_sum/use_sample_size
    theta_var = theta_square_sum/use_sample_size-theta_mean**2
    rmse = np.sqrt(((x_mean - x_truth)**2).sum()/50)
    
    THETA.append(theta_mean)
    RMSE.append(rmse)



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 534.24it/s]
